In [1]:
import numpy as np
from copy import deepcopy

In [2]:
def cost_function(board):
    cost: int = 0
    
    for i in range(9):
        # columns
        cost += (9 - np.unique(board[:,i]).size)
        # rows
        cost += (9 - np.unique(board[i, :]).size)
    
    return cost

# temperature function
def temp_function(k):
    return 1/np.exp(k/10000)

# probability that we take solution even if it is worse
def acceptance_function(T, energy_difference):
    if energy_difference < 0:
        return 1 # always go down a gradient if found a way
    return 1/(1 + np.exp(energy_difference/T))

In [3]:
def fill_board(board):
    for i in range(9):
        # get entries ont present in 3x3 block
        not_present = [el for el in range(1, 10) if el not in np.unique(board[i//3 * 3 : i//3 * 3 + 3, i % 3 * 3 : i % 3 * 3 + 3])]
        counter = 0
        for j in range(9):
            if board[i//3 * 3 + j // 3, i % 3 * 3 + j % 3] == 0:
                board[i//3 * 3 + j // 3, i % 3 * 3 + j % 3] = not_present[counter]
                counter += 1
                
    return board

In [4]:
def swap_cells(board, initial_board, swap_positions = None):
    # if positions is None we will generate them randomly
    
    if swap_positions == None:
        # sqaure index
        i = np.random.randint(0, 9, 1)
        # position in square
        j = np.random.randint(0, 9, 2)

        swap_positions = [(i//3 * 3 + j[0] // 3, i % 3 * 3 + j[0] % 3), 
                          (i//3 * 3 + j[1] // 3, i % 3 * 3 + j[1] % 3)]
    
    while initial_board[swap_positions[0]] != 0 or initial_board[swap_positions[1]] != 0:
                
        # sqaure index
        i = np.random.randint(0, 9, 1)
        # position in square
        j = np.random.randint(0, 9, 2)

        swap_positions = [(i//3 * 3 + j[0] // 3, i % 3 * 3 + j[0] % 3), 
                          (i//3 * 3 + j[1] // 3, i % 3 * 3 + j[1] % 3)]
            
    board[swap_positions[0]], board[swap_positions[1]] = board[swap_positions[1]], board[swap_positions[0]]
    
    return swap_positions[0], swap_positions[1]

In [5]:
# main function
def annealing(board, max_steps = 10000, costs = []):
    correct_board = deepcopy(board)
    fill_board(correct_board)
    T: float = -1
    current_energy: float = cost_function(correct_board)
    next_energy: float = -1
    best_energy: float = current_energy
    best_board = correct_board
    for k in range(max_steps):
        T = temp_function(k)
        swap_positions: Tuple[int, int] = swap_cells(correct_board, board)
        next_energy = cost_function(correct_board)

        # if new state not accepted swap back
        if not (acceptance_function(T, next_energy - current_energy) > np.random.uniform(0, 1)):
            swap_cells(correct_board, board, swap_positions)
            # current energy stays the same
        else:
            current_energy = next_energy

        if best_energy > current_energy:
            best_energy = current_energy
            best_board = correct_board
        
        if best_energy == 0:
            print("SOLVED!!!")
            break
        
        if k % 1000 == 0:
            print(f'k: {k} energy: {best_energy}')
           
    correct_board = best_board
    return correct_board

In [6]:
board = np.array([
                       [5, 3, 0, 0, 7, 0, 0, 0, 0],
                       [6, 0, 0, 1, 9, 5, 0, 0, 0],
                       [0, 9, 8, 0, 0, 0, 0, 6, 0],
                       [8, 0, 0, 0, 6, 0, 0, 0, 3],
                       [4, 0, 0, 8, 0, 3, 0, 0, 1],
                       [7, 0, 0, 0, 2, 0, 0, 0, 6],
                       [0, 6, 0, 0, 0, 0, 2, 8, 0],
                       [0, 0, 0, 4, 1, 9, 0, 0, 5],
                       [0, 0, 0, 0, 8, 0, 0, 7, 9]
])

In [7]:
annealing(board, max_steps= 30000)

k: 0 energy: 53
k: 1000 energy: 14
k: 2000 energy: 13
k: 3000 energy: 11
k: 4000 energy: 11
k: 5000 energy: 4
k: 6000 energy: 4
k: 7000 energy: 4
k: 8000 energy: 4
k: 9000 energy: 2
k: 10000 energy: 2
SOLVED!!!


array([[5, 3, 4, 6, 7, 8, 9, 1, 2],
       [6, 7, 2, 1, 9, 5, 3, 4, 8],
       [1, 9, 8, 3, 4, 2, 5, 6, 7],
       [8, 5, 9, 7, 6, 1, 4, 2, 3],
       [4, 2, 6, 8, 5, 3, 7, 9, 1],
       [7, 1, 3, 9, 2, 4, 8, 5, 6],
       [9, 6, 1, 5, 3, 7, 2, 8, 4],
       [2, 8, 7, 4, 1, 9, 6, 3, 5],
       [3, 4, 5, 2, 8, 6, 1, 7, 9]])

In [8]:
hard_board = np.array([
                       [0, 0, 1, 0, 6, 0, 2, 0, 0],
                       [0, 0, 0, 8, 0, 7, 0, 0, 0],
                       [4, 0, 0, 0, 9, 0, 0, 0, 3],
                       [0, 1, 0, 0, 8, 0, 0, 3, 0],
                       [5, 0, 7, 2, 0, 3, 9, 0, 1],
                       [0, 9, 0, 0, 5, 0, 0, 2, 0],
                       [7, 0, 0, 0, 2, 0, 0, 0, 5],
                       [0, 0, 0, 6, 0, 8, 0, 0, 0],
                       [0, 0, 2, 0, 3, 0, 6, 0, 0]
])

In [9]:
annealing(hard_board, max_steps= 30000)

k: 0 energy: 47
k: 1000 energy: 18
k: 2000 energy: 12
k: 3000 energy: 12
k: 4000 energy: 12
k: 5000 energy: 11
k: 6000 energy: 9
k: 7000 energy: 4
k: 8000 energy: 4
k: 9000 energy: 4
k: 10000 energy: 4
k: 11000 energy: 4
k: 12000 energy: 4
k: 13000 energy: 4
k: 14000 energy: 4
k: 15000 energy: 4
k: 16000 energy: 2
k: 17000 energy: 2
k: 18000 energy: 2
k: 19000 energy: 2
k: 20000 energy: 2
k: 21000 energy: 2
k: 22000 energy: 2
k: 23000 energy: 2
k: 24000 energy: 2
k: 25000 energy: 2
k: 26000 energy: 2
k: 27000 energy: 2
k: 28000 energy: 2
k: 29000 energy: 2


array([[8, 5, 1, 3, 6, 4, 2, 9, 7],
       [3, 2, 9, 8, 1, 7, 4, 5, 6],
       [4, 7, 6, 5, 9, 2, 1, 8, 3],
       [2, 1, 4, 9, 8, 6, 5, 3, 7],
       [5, 8, 7, 2, 4, 3, 9, 6, 1],
       [6, 9, 3, 7, 5, 1, 8, 2, 4],
       [7, 6, 8, 4, 2, 9, 3, 1, 5],
       [1, 3, 5, 6, 7, 8, 2, 4, 9],
       [9, 4, 2, 1, 3, 5, 6, 7, 8]])

**Wnioski**: dla bardziej skomplikowanych łamigłówek program znajduje minimum lokalne, nie udaje mu się jednak osiągnąć idealnego rozwiązania.